# Advanced Python <img src="images/advanced/python-logo.png" style="align: right; width: 220px" />

Before we explore the more advanced toolkits and libraries we first want to give you an overview of topics such as
- builtin python collections and when to use them
- deep dive into object-oriented programming behind the scenes,
- what are the functional programming parts of python and how to use them
- a few words about the garbage collector and finally
- tools python has for simple caching strategies

## Collections

Python comes with some builtin base collection types everybody should know about

- `tuple` - fixed size list of arbitrary items
- `list` - simple dynamic list of arbitrary items
- `set` - unordered collection of hashable values that supports mathematical operations
- `frozenset` - same as set but immutable and thus hashable
- `dict` - the default mapping type for key-value storage

However there are also more advanced lesser known collection types and helpful utilities within the python library

Overview of the classes in the collection package:
- `namedtuple` - a factory function to create tuple subclasses with named fields
- `deque` - a double-ended queue
- `ChainMap` - a proxy class to provide a common view to multiple mapping instances
- `Counter` - simple helper class for counting hashable objects
- `OrderedDict` - contrary to the general dict this class tracks insertion order and is also optimized for reordering
- `defaultdict` - dict that provides default values for keys
- `UserDict`, `UserList`, `UserStr` - base classes for implementations of dict, list or str subclasses

Also have a look at the following pages in the python documentation and the python wiki:
* __[Sequence types](https://docs.python.org/3/library/stdtypes.html#sequence-types-list-tuple-range)__
* __[Collection package docs](https://docs.python.org/3/library/collections.html)__
* __[TimeComplexity of Python Collections](https://wiki.python.org/moin/TimeComplexity)__


### Tuple

A `tuple` is a simple immutable fixed-size sequence type of arbitrary values. 

It is often used implictly (e.g. in returns), when the number of elements is fixed and ease of use is important. Since tuples are immutable they are hashable and can be used in all cases where a hashable datatype is required (e.g. keys in dicts)

In [1]:
a_tuple = (1, 'string', 32, )           # parentheses are actually optional but often used for clarity
a_tuple = tuple([1, 'string', 32, ])    # alternative creation via iterable e.g. from a list
print(a_tuple)

a, b, c = a_tuple                       # deconstruction into values
print(a, b, c)

print(f'{a_tuple[0]}, {a_tuple[2]}')    # indexed access

def calc():
    return 1, 2, 3                      # can be used to easily return values from a function

print('tuple in return value:', calc())

print('tuple hashed: ', hash(a_tuple))

(1, 'string', 32)
1 string 32
1, 32
tuple in return value: (1, 2, 3)
tuple hashed:  8337702907702286626


### List

The default python collection for storing items in a consecutive list that can dynamically grow and shrink. 

Contrary to the name, lists are internally represented as an array of references and not as e.g. a linked list. As all things in python the elements are stored indirectly (by reference) to account for the arbitrary types that can be stored. The largest runtime costs come from growing beyond the current size and inserting/deleting elements at the start because in these cases the underlying array of references needs to be resized or the references need to be moved.

A list can easily be used as a stack by using `append` and `pop` methods but if you need e.g. a FIFO queue other types are preferrable (see `deque`). The same goes for frequent lookups where other datatypes are more performant e.g. set or dict.

In [2]:
# use square brackets and comma separated values to initialize a list
b_list = ['Hello', 'World', '!']

# alternative creation via constructor using an iterable e.g. here from a tuple
b_list = list(('Hello', 'World', '!'))
print('list:', b_list)

# deconstructiong a list into variables works too
a, b, c = b_list                            
print('a, b, c: ', a, b, c)

# insert element at index
b_list.insert(2, 'Johnny')   
print('after insert:', b_list)

# append to the end
b_list.append('!')
print('after append:', b_list)

# indexed access of elements
print('indexed access:', f'{b_list[1]} {b_list[4]}')

# remove item via index
del b_list[2] 

# remove item by value
b_list.remove('World')

# remove last item
b_list.pop()
print('b_list:', b_list)

# copy whole list
c_list = b_list.copy()
print('c_list (copy):', c_list)

# reverse list
c_list.reverse()
print('c_list (reversed):', c_list)


list: ['Hello', 'World', '!']
a, b, c:  Hello World !
after insert: ['Hello', 'World', 'Johnny', '!']
after append: ['Hello', 'World', 'Johnny', '!', '!']
indexed access: World !
b_list: ['Hello', '!']
c_list (copy): ['Hello', '!']
c_list (reversed): ['!', 'Hello']


### Set (Frozenset)

If the uniqueness of items needs to be guaranteed and/or set operations such as `union`, `intersection` etc are needed, the `set` collection is your datatype of choice. In addition membership tests are also usually cheap to perform.

Internally a `set` can be thought of as an unordered hashmap so please note that a `set` can only be used with hashable items (they are not allowed to change or else the set will break). Sets also have operators such as '&', '|' and '^' defined.

A `frozenset` is basically the same as `set` but it is immutable so that it can be used as a key for e.g. a dictionary. However performance-wise they are equivalent.

In [3]:
a_set = set('Hello World!'.upper())
print(a_set)

b_set = set('How are you today?'.upper())
print(b_set)

elements_in_both = a_set.intersection(b_set)
print('elements in both sets: ', elements_in_both)

elements_in_a_but_not_in_b = a_set.difference(b_set)
print('elements in a but not in b: ', elements_in_a_but_not_in_b)

elements_in_b_but_not_in_a = b_set.difference(a_set)
print('elements in b but not in a: ', elements_in_b_but_not_in_a)

all_elements = a_set.union(b_set)
print('all elements: ', all_elements)

# demonstrate that e.g. lists cannot be added to a set
try:
    set(([1, 2, 3], [4, 5, 6]))
except Exception as e:
    print('Cannot add unhashable items - ', e)

{'!', ' ', 'O', 'H', 'L', 'D', 'E', 'R', 'W'}
{'Y', 'A', 'T', ' ', 'O', 'U', 'H', 'D', 'E', '?', 'R', 'W'}
elements in both sets:  {' ', 'O', 'H', 'D', 'E', 'R', 'W'}
elements in a but not in b:  {'!', 'L'}
elements in b but not in a:  {'A', 'T', 'U', '?', 'Y'}
all elements:  {' ', 'O', 'U', 'H', 'L', 'E', '!', 'Y', 'A', 'T', 'D', '?', 'R', 'W'}
Cannot add unhashable items -  unhashable type: 'list'


### Dict

Dicts are also a basic datatype in python. It contains unique keys that point to a certain value. Lookup via key is usually fast - so membership tests & retrieving a value is considerably cheap. Again a key needs to be hashable so that it can be used in a `dict`.

In [4]:
# initialize from kwargs
a_dict = dict(a=1, b=2)

# initialize with {}
a_dict = {'a': 1, 'b': 2}           
print(a_dict)

# update dict from another dict
a_dict.update({'b': 3, 'c': 4})     
print(a_dict)

# insert a new key-value pair
a_dict['d'] = 5                     

# change an existing value
a_dict['a'] -= 1                    
print(a_dict)

{'a': 1, 'b': 2}
{'a': 1, 'b': 3, 'c': 4}
{'a': 0, 'b': 3, 'c': 4, 'd': 5}


### Namedtuple

Named tuples were primarily created to improve readability. Behind the scenes it is actually a factory function that creates a new type with the fields given as arguments and the `tuple` type as base class. The implementation of the new type makes sure that all necessary values are there upon initialization and that an access to a non-existing element raises an exception. It supports all the operations a regular `tuple` would plus access to the attributes by name.

Regarding performance a `namedtuple` is usually always worse than a regular `tuple` since there is additional code running to realise the checks behind the scenes.

In [5]:
from collections import namedtuple
from typing import NamedTuple

# create a named tuple by using the factory function `namedtuple`
TupleFG = namedtuple('TupleFG', ('f', 'g'))
print('Inheritance chain: ', TupleFG.mro())

x = TupleFG(1, 2)
print('Instance of TupleFG: ', x)

# alternative way of creating a named tuple type
# this way it is possible to easily add type hints for the IDE
class TupleXY(NamedTuple):
    x: int
    y: int
    
y = TupleXY(3, 4)
print('Instance of TupleXY: ', y)

# use the factory function to create a named tuple
z = TupleXY._make((1, 2,))
print('Instance of TupleXY via factory function: ', z)

Inheritance chain:  [<class '__main__.TupleFG'>, <class 'tuple'>, <class 'object'>]
Instance of TupleFG:  TupleFG(f=1, g=2)
Instance of TupleXY:  TupleXY(x=3, y=4)
Instance of TupleXY via factory function:  TupleXY(x=1, y=2)


### Deque

The name `deque` stands for double ended queue and already tells a lot about the desired use case of this class. If there is the need to quickly insert and remove from the beginning/end of the data structure this collection can be used (FIFO / LIFO). It provides the necessary methods and is optimized for this very purpose.

In addition to that a `deque` can also be used with a `maxlen` argument. This means that when new items are added to one side, the collection discards as many items from the other side to satisfy the `maxlen` count again.

In [6]:
from collections import deque

a_deque = deque()

# append on the right side
a_deque.append(1)                       

# append single element on the left side
a_deque.appendleft(10)                  
print(a_deque)

# extend from iterable on the left side
# note: adds the iterable in reverse
a_deque.extendleft((4, 5, 6, ))         
print(a_deque)                          

# pop an item from the left isde
a_deque.popleft()

# pop an item from the right side
a_deque.pop()                           
print(a_deque)

# rotate elements right by 1 element
a_deque.rotate(1)                       
print(a_deque)

# create a deque with maxlength of 3
b_deque = deque(maxlen=3)
b_deque.extend((1, 2, 3, ))
print(b_deque)

# appending on the right side shifts out an element from the beginning
b_deque.append(4)                       
print(b_deque)

# appending on the left side shifts out an element from the end
b_deque.appendleft(10)                  
print(b_deque)

deque([10, 1])
deque([6, 5, 4, 10, 1])
deque([5, 4, 10])
deque([10, 5, 4])
deque([1, 2, 3], maxlen=3)
deque([2, 3, 4], maxlen=3)
deque([10, 2, 3], maxlen=3)


Find a simple performance comparison between list & deque below

In [7]:
def fifo_list():
    l = []
    for i in range(1000):
        l.append(i)
    while l:
        item = l[0]
        del l[0] 
        
print('FIFO with list')
%timeit -n 100 -r 5 fifo_list()
        
def fifo_deque():
    d = deque()
    for i in range(1000):
        d.append(i)
    while d:
        item = d.popleft()
    
print('FIFO with deqeue')
%timeit -n 100 -r 5 fifo_deque()

def lifo_list():
    l = []
    for i in range(1000):
        l.append(i)
    while l:
        item = l.pop()
        
print('LIFO with list')
%timeit -n 100 -r 5 lifo_list()

def lifo_deque():
    d = deque()
    for i in range(1000):
        d.append(i)
    while d:
        item = d.pop()
    
print('LIFO with deqeue')
%timeit -n 100 -r 5 lifo_deque()

FIFO with list
203 µs ± 1.18 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
FIFO with deqeue
86 µs ± 1.01 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
LIFO with list
86.1 µs ± 1.54 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
LIFO with deqeue
86 µs ± 1.37 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)


### ChainMap

A `ChainMap` can be used to virtually link multiple dictionaries together without discarding the original dicts. If a value gets updated it stores the updates in the first dict argument, lookups however search the full chain (from left to right).

In [8]:
from collections import ChainMap

# create three different dictionaries for this example
runtime_arguments = {}
cmdline_arguments = {'b': False, 'c': 6.2830}
default_arguments = {'a': 0, 'b': True, 'c': 3.1415}
print('runtime_arguments:', runtime_arguments)        
print('cmdline_arguments:', cmdline_arguments)
print('default_arguments:', default_arguments)

# put them in a ChainMap (order matters)
c_map = ChainMap(runtime_arguments, cmdline_arguments, default_arguments)
print('ChainMap: ', c_map)

# lookup keys and see how the value resolution works
print('param a (from default_arguments):', c_map['a'])
print('param b (from cmdline_arguments):', c_map['b'])
print('param c (from cmdline_arguments):', c_map['c'])

# adding an override via ChainMap for "c" sets the value in "runtime_arguments"
c_map['c'] = 10.0
print(c_map)

print('runtime_arguments:', runtime_arguments)        
print('cmdline_arguments:', cmdline_arguments)
print('default_arguments:', default_arguments)

print('param c (now from runtime_arguments):', c_map['c'])

runtime_arguments: {}
cmdline_arguments: {'b': False, 'c': 6.283}
default_arguments: {'a': 0, 'b': True, 'c': 3.1415}
ChainMap:  ChainMap({}, {'b': False, 'c': 6.283}, {'a': 0, 'b': True, 'c': 3.1415})
param a (from default_arguments): 0
param b (from cmdline_arguments): False
param c (from cmdline_arguments): 6.283
ChainMap({'c': 10.0}, {'b': False, 'c': 6.283}, {'a': 0, 'b': True, 'c': 3.1415})
runtime_arguments: {'c': 10.0}
cmdline_arguments: {'b': False, 'c': 6.283}
default_arguments: {'a': 0, 'b': True, 'c': 3.1415}
param c (now from runtime_arguments): 10.0


### Counter

A `Counter` class can be used to quickly count the occurences of (hashable) items in a list. It works like a dict, where the keys are the items and the values are the counts.

In [9]:
from collections import Counter

cnt1 = Counter('Hello World!')                      # feed in a string (=iterable) and count the occurences of each character
print('cnt1: ', cnt1)
print(cnt1.most_common(1))                          # print out the most common character

cnt2 = Counter({'red': 3, 'blue': 1, 'green': 0})   # can also take in a mapping that already has counts
print('cnt2: ', cnt2)
print(cnt2.most_common(1))
print(cnt2.total())

cnt3 = Counter(('green', 'blue', 'yellow', 'blue'))
print('cnt3: ', cnt3)

cnt4 = cnt2 + cnt3                                  # combine two counters
print('cnt4: ', cnt4)


cnt1:  Counter({'l': 3, 'o': 2, 'H': 1, 'e': 1, ' ': 1, 'W': 1, 'r': 1, 'd': 1, '!': 1})
[('l', 3)]
cnt2:  Counter({'red': 3, 'blue': 1, 'green': 0})
[('red', 3)]
4
cnt3:  Counter({'blue': 2, 'green': 1, 'yellow': 1})
cnt4:  Counter({'red': 3, 'blue': 3, 'green': 1, 'yellow': 1})


### OrderedDict

`OrderedDict` is very different from the default `dict` implementation. It shines when insertion order is crucial and frequent reordering operations are necessary. However it has considerable drawbacks when space efficiency, iteration speed and update operations is of importance - in these cases a standard `dict` should be used instead.

In [10]:
from collections import OrderedDict

# create a new ordered dict form a dict argument
o_dict = OrderedDict({'c': 10, 'a': 2, 'b': 3})
print(o_dict.keys())

# move the key 'c' to the end of the ordered dict
o_dict.move_to_end('c')             
print(o_dict.keys())

# remove an item from the back
o_dict.popitem()                    
print(o_dict.keys())

odict_keys(['c', 'a', 'b'])
odict_keys(['a', 'b', 'c'])
odict_keys(['a', 'b'])


### Defaultdict

`defaultdict` is a subclass of `dict` and behaves just like regular dictionaries. The main difference is that you can provide a factory function for values. Whenever a key is not yet in the dictionary it is created using the default factory function.

In [11]:
from collections import defaultdict

a_dict = defaultdict(lambda: 100)
print(a_dict.keys())
a_dict['a'] += 10
print(a_dict['a'])
print(a_dict.keys())
a_dict['b'] -= 40
print(a_dict['b'])
print(a_dict.keys())
print(a_dict)

b_dict = defaultdict(list)
for name, value in [('foo', 'bar'), ('foo', 'baz'), ('bar', 'boo')]:
    b_dict[name].append(value)
print(b_dict)


dict_keys([])
110
dict_keys(['a'])
60
dict_keys(['a', 'b'])
defaultdict(<function <lambda> at 0x7f134ae77520>, {'a': 110, 'b': 60})
defaultdict(<class 'list'>, {'foo': ['bar', 'baz'], 'bar': ['boo']})


### UserDict, UserList, UserStr

These classes exist to simplify user defined implementations of `dict`, `list` as well as `str`. They make their content available in the `data` attribute and thus allow operating on the data for e.g. custom implementations of existing functions or new functions.

In [12]:
from collections import UserList

class MyUserList(UserList):    
    def clear_odd(self):
        """ Add a simple custom function to clear the add values (assumes that the items are int). """
        self.data = [v for v in self.data if v % 2 == 0]
        
l = MyUserList([1, 2, 3, 4, 5, 6, 7, 8])
print('Initial list:', l)
l.clear_odd()
print('After executing custom method: ', l)

Initial list: [1, 2, 3, 4, 5, 6, 7, 8]
After executing custom method:  [2, 4, 6, 8]


### Dataclasses

Dataclasses where introduced in Python 3.7 and allow the user to quickly put together simple data container classes with type hints. 

The supported methods can be customized by specifying flags in the decorator.

Regarding performance and memory requirements dataclasses can be as fast as slotted classes (depending on the creation arguments in the decorator).

In [13]:
from dataclasses import dataclass
from typing import Any

@dataclass(repr=False, eq=False, order=False, slots=True)
class DataclassDemo:
    x: int
    y: int
    value: Any
    
data = DataclassDemo(10, 12, dict())


## Object-oriented Programming

Also see: __[Python Datamodel](https://docs.python.org/3/reference/datamodel.html)__

A class in python is simply put a template object (a type) to create a class instance comprised of certain attributes & methods to operate on these attributes.

The most simple way a class can be defined is like this `class C: pass`. The previous code is actually equivalent to the following `C = type('C', (), dict())`

### Special attributes and methods

When you create a new instance of the `Test` class python first calls the `__new__` method, which is implicitly a static method of the class type itself. `__new__`'s job is to actually create a new object instance and then call the `__init__` method on the instance.

Internally attributes are saved in the private attribute `__dict__`.

Whenever the official string representation of class instance is needed or when `repr` is directly called, the method `__repr__` is invoked on the instance to generate this string. If there is no `__str__` method defined `__repr__` is also used to generate the informal string representation of the instance.

Other often used special methods are the comparison methods (`__lt__`, `__eq__`, `__gt__`, etc ...) the hash method `__hash__` and the attribute access method (`__getattr__`, `__getattribute__`, `__setattr__`, ...). Please refer to the python documentation for a complete list and description of those methods.

In [14]:
# create a new class
class Test:
    def __new__(cls, *args, **kwargs):
        print('new instance: __new__')
        return super().__new__(cls)
        
    def __init__(self, *args, **kwargs):
        print('new instance: __init__')
        super().__init__()
        self.test_attribute = 10
        
    def __repr__(self):
        return 'Test()'
    
    
# create a new instance of `Test` class
t = Test()
print(t)
print(t.__dict__)
print(t.__hash__())

new instance: __new__
new instance: __init__
Test()
{'test_attribute': 10}
8732552570355


Attributes, Methods and most of the charcteristics of a class can be changed at runtime.

In [15]:
# print class name
print('Class name: ', t.__class__.__name__)

# dict is empty
print('Empty class instance: ', t.__dict__)

# dynamically add a new attribute
t.new_attribute = 'foo'

# dict contains 'new_attribute' afterwards
print('Class instance afterwards: ', t.__dict__)

# print default representation
print('Default repr of class instance: ', repr(t))

# change __repr__ method at runtime for all instances
Test.__repr__ = lambda self: f'<Test new_attribute={self.new_attribute}>'     

# print new representation
print('New repr of class instance: ',repr(t))

# dynamically replace constructor
def new_init(self, other_attribute):
    self.other_attribute = other_attribute
    
Test.__init__ = new_init

# create new instance using new constructor
u = Test('other')

# dict contains 'other_attribute'
print('With new constructor: ', u.__dict__)

try:
    # creating a new instance now requires the new attribute
    Test()
except TypeError:
    print('Caught TypeError after setting new "__init__" method')

Class name:  Test
Empty class instance:  {'test_attribute': 10}
Class instance afterwards:  {'test_attribute': 10, 'new_attribute': 'foo'}
Default repr of class instance:  Test()
New repr of class instance:  <Test new_attribute=foo>
new instance: __new__
With new constructor:  {'other_attribute': 'other'}
new instance: __new__
Caught TypeError after setting new "__init__" method


### Slots

If arbitrary attributes are not necessary a class can be restricted to only allow certain attributes by using so called slots. In addition to having a check in place that an attribute is really existing it can also save memory and improve lookup speeds because the class does not need to have an internal `dict` to store possible additional attributes.

The attribute lookup usually works by taking the attribute name and translating it into a lookup into the internal `__dict__`. Using slots creates descriptors for those attributes and prevents python from creating the `__dict__` attribute.

In [16]:
from sys import getsizeof
from tooling.getsize import getsize

class ClassExample:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class_example = ClassExample(1, 2, 3)
print('Size of class with __dict__: ', getsizeof(class_example))
print('Actual size of class with __dict__: ', getsize(class_example))


class SlotExample:
    __slots__ = ('x', 'y', 'z')
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z       

slot_example = SlotExample(1, 2, 3)
print('Size of class with __slots__: ', getsizeof(slot_example))
print('Actual size of class with __dict__: ', getsize(slot_example))

try:
    slot_example.new_attribute = 'bar'
except AttributeError:
    print('Caught AttributeError - assignment of new attributes is not possible when slots are used.')

Size of class with __dict__:  48
Actual size of class with __dict__:  236
Size of class with __slots__:  56
Actual size of class with __dict__:  140
Caught AttributeError - assignment of new attributes is not possible when slots are used.


### Inheritance 

Python also supports polymorphism and (multiple) inheritance. The inheritance hierarchy is defined by the MRO (method resolution order) and can be looked up at runtime by using the method `mro`. 

Regarding method resolution order - consider the following example:

In [17]:
class Animal: 
    def speak(self):
        raise NotImplementedError

class Dog(Animal): 
    def speak(self):
        return 'bark'
    
class Cat(Animal):
    def speak(self):
        return 'meow'

class DogCat(Dog, Cat):
    pass

kotpies = DogCat()
print('MRO of C: ', type(kotpies).mro())

# use 'isinstance' to check if an instance is of a given type
print('is "kotpies" an instance of Animal: ', isinstance(kotpies, Animal)) 
print('is "kotpies" an instance of Dog: ', isinstance(kotpies, Dog))    

print('kotpies says:', kotpies.speak())

MRO of C:  [<class '__main__.DogCat'>, <class '__main__.Dog'>, <class '__main__.Cat'>, <class '__main__.Animal'>, <class 'object'>]
is "kotpies" an instance of Animal:  True
is "kotpies" an instance of Dog:  True
kotpies says: bark


From the above example we can see that the MRO works by going from subclass(es) to baseclass until it stops at the implicit `object` base class every python type inherits from. Note that the order of the classes in the definition of `DogCat` matters (`Dog` before `Cat`).

Classes also can have static attributes, static methods and class methods.

In [18]:
class Static:
    count = 10
    
    @staticmethod
    def static_method():
        return f'static count is: "{Static.count}"'
    
    @classmethod
    def class_method(cls):
        return f'class: "{cls.__name__}" bar - static count is "{cls.count}"'
    
a, b = Static(), Static()
print(a.count, b.count, Static.count)

Static.count = 5
print(a.count, b.count, Static.count)

# however be aware! when trying to access a static attribute like this, it creates a new instance attribute instead
a.count = 4                                 
print(a.count, b.count, Static.count)

# call the static method, it has access to static attributes
print(Static.static_method())

# call the class method, which is similar to static method but gets the type as first argument
print(Static.class_method())

# you can also call static and class methods via an instance
print(a.static_method())
print(b.class_method())

10 10 10
5 5 5
4 5 5
static count is: "5"
class: "Static" bar - static count is "5"
static count is: "5"
class: "Static" bar - static count is "5"


### Metaclasses / Metaprogramming

In python everything is an object - even types themselves are objects. So creating a class derived from `type` can be used to add additional behaviour to instances created by this class. This class that creates other class instances is called a `metaclass`. 

This kind of mechanism is often used e.g. to realize ORM systems or to generate additional methods or code for definitions in classes. 

Note that every class in python can only have one metaclass.

In [19]:
class TheMetaclass(type):
    def __new__(cls, name, bases=None, namespace=None):
        print('using "TheMetaclass"', name, bases)
        for key, value in namespace.items():
            print(f'{key}: {value}')
        return type.__new__(cls, name, bases, namespace)

class TheClass(metaclass=TheMetaclass):
    blub = dict()

c = TheClass()
print(c)

using "TheMetaclass" TheClass ()
__module__: __main__
__qualname__: TheClass
blub: {}


## Functional Programming

**Functional programming** can be a very powerful tool in python. The basic idea is that we use functions as if they were mathematical transformations that are applied to data. Thus we can easily stack them together to produce the results we want. 

### Generators

To understand how this works in python it is necessary to first understand generator functions: A generator function is a special function that does not just return a value but instead returns an **iterator** which can be used to process a sequence of values. This is done by using `yield` instead of `return` to return the `next` result. 

If big amounts of input data, that do not fit into memory, need to be processed, the concept of generator functions can be very useful as it allows processing a value (or a chunk of values) at a time (stream processing).

In [20]:
def square(input_sequence):
    for item in input_sequence:
        yield item*item

# use the list constructor to evaluate the sequence for printing
data = (1, 2, 3, 4, )
print(list(square(data)))

[1, 4, 9, 16]


### Using built-in generator functions

In the above example we could already see how such a generator function can be used to square the input data, yielding one result at a time. Python already comes with a lot of built-in functions you probably already used that are actually generator functions - for example think about `range`. Other often used examples are `all`, `any`, `filter`, `map`, `max`, `min`, `reversed`, `sorted`, `sum` and `zip`.

Compared to simply looping over the data in python code, the build-in function can also quite considerably lead to speed-ups because they do not need to be interpreted first.

In [21]:
import dis

# create a list of test data
test_data = []
for i in range(10000):
    test_data.append(i)

In [22]:
def sum_data_loop(input_list):
    s = 0
    for element in input_list:
        s += element
    return s

print('sum data using for loop')
%timeit -n 1000 -r 3 sum_data_loop(test_data)
print(sum_data_loop(test_data))
dis.dis(sum_data_loop)

sum data using for loop
354 µs ± 1.96 µs per loop (mean ± std. dev. of 3 runs, 1,000 loops each)
49995000
  2           0 LOAD_CONST               1 (0)
              2 STORE_FAST               1 (s)

  3           4 LOAD_FAST                0 (input_list)
              6 GET_ITER
        >>    8 FOR_ITER                 6 (to 22)
             10 STORE_FAST               2 (element)

  4          12 LOAD_FAST                1 (s)
             14 LOAD_FAST                2 (element)
             16 INPLACE_ADD
             18 STORE_FAST               1 (s)
             20 JUMP_ABSOLUTE            4 (to 8)

  5     >>   22 LOAD_FAST                1 (s)
             24 RETURN_VALUE


In [23]:
def sum_data_fn(input_list):        # use a function so we can disassemble it
    return sum(input_list)

print('sum data using "sum" function')
%timeit -n 1000 -r 3 sum_data_fn(test_data)
print(sum_data_fn(test_data))
dis.dis(sum_data_fn)

sum data using "sum" function
82 µs ± 365 ns per loop (mean ± std. dev. of 3 runs, 1,000 loops each)
49995000
  2           0 LOAD_GLOBAL              0 (sum)
              2 LOAD_FAST                0 (input_list)
              4 CALL_FUNCTION            1
              6 RETURN_VALUE


It can be seen that in the first function `sum_data_loop` the interpreter needs to go through all of the instructions which is a lot more compared to the second function `sum_data_fn` which simply calls a built-in C-function behind the scenes.

The complete list of built-in functions can be found here: __[Built-in Functions](https://docs.python.org/3/library/functions.html)__.

### Lambdas

When using functional programming programmers often run into the problem that they need to define functions that can be applied to e.g. `map` or `filter`.

In [24]:
def is_even(number):
    return number % 2 == 0

a_list = [1, 2, 3, 4]
print(list(filter(is_even, a_list)))

[2, 4]


Instead of first definining and using the method `is_even` like above we can also just write a so-called **lambda expressoin**. 

Lambdas are in-place functions that are defined by using the syntax `lambda [list of arguments]: <expression>`. 

Since lambdas are itself objects, they can also be assigned and used multiple times if desired.

In [25]:
a_list = [1, 2, 3, 4]
print(list(filter(lambda number: number % 2 == 0, a_list)))

is_even_lambda = lambda number: number % 2 == 0
b_list = [4, 5, 6, 7, 8, 9, 10]
print(list(map(is_even_lambda, b_list)))

[2, 4]
[True, False, True, False, True, False, True]


In [26]:
%timeit -n 100000 -r 3 list(filter(is_even, b_list))


767 ns ± 1.04 ns per loop (mean ± std. dev. of 3 runs, 100,000 loops each)


In [27]:
%timeit -n 100000 -r 3 list(filter(lambda number: number % 2 == 0, b_list))


832 ns ± 6.58 ns per loop (mean ± std. dev. of 3 runs, 100,000 loops each)


In [28]:
%timeit -n 100000 -r 3 list(filter(is_even_lambda, b_list))


795 ns ± 5.78 ns per loop (mean ± std. dev. of 3 runs, 100,000 loops each)


It is worth noting that a function and a lambda actually produce the same code. Depending on the use-case it can however help to pre-define to lambda to omit multiple evaluations.


In [29]:
import dis
print('function')
dis.dis(is_even)
print('lambda')
dis.dis(is_even_lambda)

function
  2           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 (2)
              4 BINARY_MODULO
              6 LOAD_CONST               2 (0)
              8 COMPARE_OP               2 (==)
             10 RETURN_VALUE
lambda
  4           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 (2)
              4 BINARY_MODULO
              6 LOAD_CONST               2 (0)
              8 COMPARE_OP               2 (==)
             10 RETURN_VALUE


### Putting it together

Now that the basics have been explained lets have a look at a more complicated example and see how functional programming can help us here. 

Suppose we want to write a function that creates a list of numbers from 0 to 9999, squares them and filters out the odd numbers. We could start with the following code:

In [30]:
max_numbers = 10000

def get_even_squared_numbers():     
    input_numbers = []
    for i in range(max_numbers):
        input_numbers.append(i)
    
    squared_numbers = []
    for i in input_numbers:
        i_squared = i**2
        squared_numbers.append(i_squared)
        
    even_squared_numbers = []
    for i in squared_numbers:        
        if i % 2 == 0:
            even_squared_numbers.append(i)
    
    return even_squared_numbers
    
%timeit -n 100 -r 5 get_even_squared_numbers()

3.76 ms ± 8.44 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)


How could the above code be rewritten into a chain of functions?

In [31]:
max_numbers = 10000

%timeit -n 200 -r 3 list(filter(lambda i: i % 2 == 0, map(lambda i: i**2, range(max_numbers))))

3.62 ms ± 6.45 µs per loop (mean ± std. dev. of 3 runs, 200 loops each)


There are also other helpful tools for functional programming in the `functools` package e.g. binding arguments. 

With this we can also rewrite the example from above to use e.g. `pow` by binding the argument 

In [32]:
from functools import partial

max_numbers = 10000

pow_two = partial(pow, exp=2)

%timeit -n 200 -r 3 list(filter(lambda i: i % 2 == 0, map(pow_two, range(max_numbers))))

4.66 ms ± 10.6 µs per loop (mean ± std. dev. of 3 runs, 200 loops each)


Please note that the runtime actually gets longer after we started using the partial function and the built in `pow` but this has nothing to do with `partial` - its simply that the implementation of `pow` is much slower as the previously used lambda.

So can the above code be further optimized (apart from the pow)?

#### Further improvements


Yes, definitely! 

What actually happens when you define a `lambda` and pass it into the `filter` function is that a lambda expression gets created and evaluated as a function. This function is then passed into the `filter` function and used to filter the input sequence. So by defining them up-front we can save this cost. 

However this is not even the major part of where the runtime is spent. The culprit is actually the `**` operator which is much slower (and generic) than simply multiplying the two values.

The last thing we can improve should actually have been the most obvious thing: Instead of filtering after the costly operation we could filter the values up-front since even values were even before the are squared as well.

In [33]:
max_numbers = 10000
pow_two = lambda i: i ** 2
is_even = lambda i: i % 2 == 0
%timeit -n 200 -r 3 list(filter(is_even, map(pow_two, range(max_numbers))))

3.63 ms ± 2.58 µs per loop (mean ± std. dev. of 3 runs, 200 loops each)


In [34]:
max_numbers = 10000
pow_two = lambda i: i * i
is_even = lambda i: i % 2 == 0
%timeit -n 500 -r 3 list(filter(is_even, map(pow_two, range(max_numbers))))

1.57 ms ± 1.13 µs per loop (mean ± std. dev. of 3 runs, 500 loops each)


In [35]:
max_numbers = 10000
pow_two = lambda i: i * i
is_even = lambda i: i % 2 == 0
input_data = list(range(max_numbers))
%timeit -n 500 -r 3 list(map(pow_two, filter(is_even, input_data)))

1.14 ms ± 933 ns per loop (mean ± std. dev. of 3 runs, 500 loops each)


### List comprehensions

Instead of using functions python also offers the possibility to use so called **list comprehensions**. This is a special python syntax that allows for easy transformation of collections - most of the time its also one of the fastest options. 

In [36]:
# create a list as data input
my_list = ['Hello', 'World', 'How', 'are', 'you', '?']
print(my_list)

# use list comprehension to filter the list
my_filtered_list = [element 
                    for element in my_list 
                    if 'o' in element]
print(my_filtered_list)

# use for mapping map elements into new elements
word_lenghts = [len(element) for element in my_list]
print(word_lenghts)

# list comprehensions can also produce dictionaries
as_dict = {element: len(element) for element in my_list}
print(as_dict)

# or you can use a dictionary as input 
calculated_values = {key: value*value for key, value in as_dict.items()}
print(calculated_values)

['Hello', 'World', 'How', 'are', 'you', '?']
['Hello', 'World', 'How', 'you']
[5, 5, 3, 3, 3, 1]
{'Hello': 5, 'World': 5, 'How': 3, 'are': 3, 'you': 3, '?': 1}
{'Hello': 25, 'World': 25, 'How': 9, 'are': 9, 'you': 9, '?': 1}


List comprehensions can also be nested. See the following example where the first `for` iterates over the items of the outer list (the `sublist`) and the second `for` then iterates over the elements in the sublist.

In [37]:
nested_list = [[1, 2, 3, 4, 5],
               [4, 3, 1],
               [1, 3]]
print(nested_list)

flat_list = [element              
             for sublist in nested_list
             for element in sublist]
print(flat_list)

[[1, 2, 3, 4, 5], [4, 3, 1], [1, 3]]
[1, 2, 3, 4, 5, 4, 3, 1, 1, 3]


Lets take the example from earlier with all the optimizations applied and rewrite it using a list comprehension:

In [38]:
max_numbers = 10000
input_data = list(range(max_numbers))
%timeit -n 500 -r 3 [i*i for i in input_data if i % 2 == 0]

568 µs ± 718 ns per loop (mean ± std. dev. of 3 runs, 500 loops each)


Please note: even though it can be very elegant and even faster than (or equal to) functional programming there are also situations where you just simply cannot use list comprehensions because the control flow limitations don't allow it. So always consider all your options!

# Garbage Collection

Python uses a garbage collector behind the scenes to keep track of its used memory. 

The algorithm mainly relies on reference counting which already works very well on its own. However with reference counting alone the garbage collector would not be able to detect reference cycles or references in unreachable objects (once an object that holds a reference to another object becomes unreachable the count of the referenced object will never go down to zero). 

To be able to collect these references as well python also has an additional cyclically running generational garbage collector (see __[Garbage Collector Design](https://devguide.python.org/internals/garbage-collector/)__).

Python objects do come with a `__del__` method but be aware: **this method will only be called when the instance actually gets garbage collected**, so this is not a reliable way to clean up any resources!

In [39]:
import sys

class A:    
    def __init__(self):
        print(self, 'created')
        
    def __del__(self):                      # __del__ is a special method that gets called when an object instace is collected by the GC
        print(self, 'deleted')    

In [40]:
reference = A()
print('rc:', sys.getrefcount(reference))    # instance created and referenced by 'reference' and by passing it to getrefcount -> 2

second_reference = reference
print('rc:', sys.getrefcount(reference))    # instance referenced by reference, second_reference and in getrefcount -> 3

l = [reference]
print('rc:', sys.getrefcount(reference))    # reference appended to list -> 4

del l[0]
print('rc:', sys.getrefcount(reference))    # element deleted from list -> 3

del second_reference                    
print('rc:', sys.getrefcount(reference))    # explicitly removed the second_reference -> 2

del reference                               # remove last reference -> 0 (object gets collected, __del__ gets called)

<__main__.A object at 0x7f134b194550> created
rc: 2
rc: 3
rc: 4
rc: 3
rc: 2
<__main__.A object at 0x7f134b194550> deleted


The garbage collector module `gc` provides access to garbage collector settings and its also possible to explicitly trigger garbage collection runs or to disable automatic garbage collection through it.

In [41]:
import gc

# retrieve the thresholds of each GC generation
print('Generation thresholds: ', gc.get_threshold())

# disable automatic garbage collection
gc.disable()    

# run a full collection over all generations, manually
# result returned is the number of unreachable objects found
result = gc.collect()    
print(result)

# enable automatic garbage collection again
gc.enable()

Generation thresholds:  (700, 10, 10)
230


As mentioned earlier the garbage collector can detect cyclic references but needs a full run to do so. See the following example for a demonstration.

In [42]:
import gc
import sys

c1 = A()
c2 = A()
c1.ref = c2
c2.ref = c1

# object references itself (+1 temp from getrefcount), so ref count is -> 3
print('rc:', sys.getrefcount(c1), sys.getrefcount(c2))

# deleting the references decreases the ref count only to 1
# they are still holding a reference to each other and thus the instances
# cannot be collected that easily
del c1
del c2

# we can however explictly call the garbage collector to collect the instances 
# (or instead wait for python to do the same on its own)
print('explicit call to gc')
print('gc result:', gc.collect())

<__main__.A object at 0x7f134b30d570> created
<__main__.A object at 0x7f134b30d690> created
rc: 3 3
explicit call to gc
<__main__.A object at 0x7f134b30d570> deleted
<__main__.A object at 0x7f134b30d690> deleted
gc result: 4


In order to make cleanup more deterministic in terms of memory usage and releasing resources, it can be a good strategy (depending on the problem) to define methods which can be used to trigger specific cleanup actions e.g. close ressources such as files or devices and/or delete references to other instances.

This way the programmer can use the reference counting mechanism to his advantage to make python collect object instances whenever its possible.

In [43]:
import sys

class A:    
    def __init__(self):
        self.ref = None
        print(self, 'created')
        
    def __del__(self):
        self.cleanup()              # make sure that cleanup gets called in every case
        print(self, 'deleted')    
        
    def cleanup(self):
        if self.ref:
            self.ref = None

# define two instances that reference each other
c1 = A()
c2 = A()
c1.ref = c2
c2.ref = c1

# printing the refcount actually shows us that both objects have a ref count of `3`
print('rc:', sys.getrefcount(c1), sys.getrefcount(c2))

# ... do stuff ...

# explicitly call cleanup to release resources at this point
c1.cleanup()
c2.cleanup()

# ref count has now dropped to `2` since the internal references have been cleaned up
print('rc:', sys.getrefcount(c1), sys.getrefcount(c2))

# deleting the last remaining references, the instances can now be collected immediately
del c1
del c2

<__main__.A object at 0x7f134b2f0c40> created
<__main__.A object at 0x7f134b2f3e50> created
rc: 3 3
rc: 2 2
<__main__.A object at 0x7f134b2f0c40> deleted
<__main__.A object at 0x7f134b2f3e50> deleted


## Caching

Python also comes with built-in tools to cache method results (also often called `memoizing`). 

Caching can be a simple and effective technique to speed up certain computations - with the trade off of using more memory. 

These tools can also be found in the already mentioned `functools` package (see __[functools](https://docs.python.org/3/library/functools.html)__).


In [44]:
import math
from functools import cache

def compute(a, b):
    return math.sqrt(pow(a, 2) + pow(b, 2)) / math.sqrt(pow(a, 3) + pow(b, 3))

@cache
def cached_compute(a, b):
    return math.sqrt(pow(a, 2) + pow(b, 2)) / math.sqrt(pow(a, 3) + pow(b, 3))

print('uncached')
%timeit -n 100000 -r 5 compute(1234, 5678)

print('cached')
%timeit -n 100000 -r 5 cached_compute(1234, 5678)

uncached
1.36 µs ± 4.45 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)
cached
99.1 ns ± 1.26 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)


Always be aware that this may take a lot of the systems memory! 

If there are a lot of possible input combinations it may be better to limit the number or stored results e.g. if only recent computation results are important it is possible to use `lru_cache` instead. LRU (last recently used) caching is slightly slower, since it needs to maintain the maxsize of the cache.

In [45]:
import math
from functools import lru_cache

@lru_cache(maxsize = 128)
def lrucompute(a, b):
    return math.sqrt(pow(a, 2) + pow(b, 2)) / math.sqrt(pow(a, 3) + pow(b, 3))

print('lru cache')
%timeit -n 100000 -r 5 lrucompute(1234, 5678)

lru cache
93.2 ns ± 1.59 ns per loop (mean ± std. dev. of 5 runs, 100,000 loops each)


## Summary

Regardless of the things presented in this chapter: **always consider your problem space, measure things and think about algorithmic performance first**

Then:
- Choose the right collection for your problem
- Use functions (e.g. sum, map) or list comprehensions instead of plain loops to speed up computations
- Look into your class / object layout 
- Consider the garbage collector if you run into ressource problems
- Think about trading memory space for speed or vice-versa